In [1]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [2]:
action=[(0,0), (0,1), (1,1)]

Number_level = 10;
# #Transition_matrix = [0.86 0.14 0.0 0.0; 0.0 0.83 0.17 0.0;0.0 0.0 0.89 0.11;1.0 0.0 0.0 0.0];
# fullname = "tm10";
# # generate_transition_matrix(Number_level, fullname) 
# Transition_matrix = load(fullname*".jld2","transition_matrix");
# failure_penalty = -1000.0;
# maintenance_penalty = -200.0;
# setup_cost = -800.0;
# system_penalty = - 2000.0;
# normal_operation = 0.0;

In [3]:
T=zeros(Number_level,Number_level,2,100);

# Load transition probability matrix for each component
for i in 1:100
    #fullname = "/Users/huhan/Library/CloudStorage/GoogleDrive-hhu98@wisc.edu/My Drive/Master's Thesis/Heterogeneous/TM_MATRIX/tm"*string(i);
    fullname = "J:/Master's Thesis/Heterogeneous/TM_MATRIX/tm"*string(i);
    # generate_transition_matrix(Number_level, fullname) 
    Transition_matrix = load(fullname*".jld2","transition_matrix");
    # the last term in argument of T, which is written as 'i' here,
    # is the index of component in the heterogeneous system
 
    T[:,:,1,i].=Transition_matrix;
    T[:,:,2,i].=Transition_matrix;
    for j in 1:10
        T[j,:,2,i].=Transition_matrix[1,:];
    end
    T[10,:,1,i].=Transition_matrix[1,:]
    T[10,:,2,i].=Transition_matrix[1,:]
end

In [4]:
T[:,:,:,1]

10×10×2 Array{Float64, 3}:
[:, :, 1] =
 0.232649  0.229122  0.130585  0.0954644  …  0.0507595  0.0416229  0.00861626
 0.0       0.218724  0.163234  0.128353      0.0899794  0.0431818  0.00868773
 0.0       0.0       0.166395  0.182476      0.111027   0.0773355  0.043199
 0.0       0.0       0.0       0.269557      0.125296   0.0981466  0.0533485
 0.0       0.0       0.0       0.0           0.125828   0.109077   0.0611228
 0.0       0.0       0.0       0.0        …  0.215491   0.17866    0.0619123
 0.0       0.0       0.0       0.0           0.352355   0.221208   0.0646198
 0.0       0.0       0.0       0.0           0.471891   0.449917   0.0781922
 0.0       0.0       0.0       0.0           0.0        0.591116   0.408884
 0.232649  0.229122  0.130585  0.0954644     0.0507595  0.0416229  0.00861626

[:, :, 2] =
 0.232649  0.229122  0.130585  0.0954644  …  0.0507595  0.0416229  0.00861626
 0.232649  0.229122  0.130585  0.0954644     0.0507595  0.0416229  0.00861626
 0.232649  0.229122  

In [5]:
# Note that the reward function here is only decided by component-wise independent knowledge, 
# i.e., we know nothing about other components when making decision for this component
function reward(s, a, s1=10, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
  
       if a==1 # don't replace component
        if Number_level>s
            # crd = Categorical(Transition_matrix[s,:]);
            # s = rand(crd);
            r = normal_operation ;
         return r
        else
            r=system_penalty/NumberUnits;
            return r
        end
        end
#     if a==2
#          if s1>=s
#             crd = Categorical(Transition_matrix[s,:]);
#             s = rand(crd);
#             r = normal_operation +  setup_cost/NumberUnits;
#          return r
#         else
#             return 0.0
#         end
#         end
if a==2 # replace component
    if Number_level>s
        # s=1
        # crd = Categorical(Transition_matrix[s,:]);
        # s = rand(crd);
        r = maintenance_penalty+  setup_cost/NumberUnits;
     return r
    else
        # s=1
        # crd = Categorical(Transition_matrix[s,:]);
        # s = rand(crd);
        r= failure_penalty  +  setup_cost/NumberUnits;
    return r
    end
    end
end

reward (generic function with 3 methods)

In [6]:
s=1:Number_level;

In [7]:
((1:10)./10)

0.1:0.1:1.0

In [8]:
function explore()
    c=[1/3,1/3,1/3]

     crd = Categorical(c);
     return rand(crd)
    end 
function generative(s, a, k, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
        if s==Number_level # corrective replacement
        s=1
        crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r=failure_penalty + normal_operation  ;
        return (sp=s, r=r)
       end
        r=0


    if a==1 # don't replace component
            if Number_level>s
                crd = Categorical(T[s,:,1,k]);
                s = rand(crd);
                r = normal_operation ;
                return (sp=s, r=r)
            else
                r=0;
                return (sp=s, r=r)
            end
            end

 

    if a==2 # replace component
        if Number_level==s
            s=1
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r= failure_penalty;
            
            return (sp=s, r=r)
        end
        if s==1
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r= 0;
            return (sp=s, r=r)
        end
        s=1
        crd = Categorical(T[s,:,1,k]);
        s = rand(crd);
        r = maintenance_penalty;
        return (sp=s, r=r)
    end
    end

generative (generic function with 2 methods)

In [9]:
# the value iteration here is simply based on independent component-level knowledge
function ValueIteration(γ, p1=0.01, d=100)
    Q=zeros((Number_level,2,NumberUnits))
#     print(Q)
    V= zeros(Float64, (Number_level,NumberUnits)).+1000
    diff=0
    i=0
    
    

    
    p=[1 1 1 ]
    for k in 1:NumberUnits
    for i in 1:10000
    for s in 1:10
        Threads.@threads for a in 1:2
 
                Q[s,a,k]= (reward(s,a)) + γ*( sum(T[s,s1,a,k]*(V[s1,k]) for (s1) in 1:10 ))
    end
            V[s,k]=findmax(Q[s,1:2,k])[1]
        end
    end
        end
    return Q,V
    end

ValueIteration (generic function with 3 methods)

In [10]:

# function decison(Q, s)
#    a_final=[]
#    if Number_level in s
# #         print("hi")
#         d=(collect(Iterators.product((Q[s[i],2:3] for i in 1:NumberUnits)...)))
#         d1=(collect(Iterators.product((2:3 for i in 1:NumberUnits)...)))
#         min=-1e+10
#         opt=0
#         for (i,j) in enumerate(d)
#             if min < sum((j))
#                 min=sum((j))
#                 opt=i
#             end
#         end
#         return d1[opt]
#     end
#     q0=0.0
#     a_0=repeat(1:1,NumberUnits);
#     for i in 1:NumberUnits
#             q0+=Q[s[i],1]
#     end
#     d=(collect(Iterators.product((Q[s[i],2:3] for i in 1:NumberUnits)...)))
#     d1=(collect(Iterators.product((2:3 for i in 1:NumberUnits)...)))
# #     d[1]=a_0
# #     print("here")
# #     d1[1]=(1,1,1,1,1,1,1,1,1,1)
#     min=-1e+10
#     opt=0
#     for (i,j) in enumerate(d)
#         if i==1
#             if min < q0
#             min=q0
#             opt=i
#         end
#         elseif min < sum((j))
#             min=sum((j))
#             opt=i
#         end
#     end
#     if opt==1
#         return a_0
#     else
#         return d1[opt]
#     end
#     return opt
# end


function decison(Q, s,n=0.2)
   a_final=[]
   if true
#         print("hi")
        for i in 1:NumberUnits
            append!(a_final,argmax(a->Q[s[i],a,i], [1,2]))
            end
        return a_final
    end
end

decison (generic function with 2 methods)

In [11]:
using DataFrames
df= DataFrame(u=[],K=[],n=[],s=[],m=[],f=[],p=[],lamb=[],mean=[],std=[])

Row,u,K,n,s,m,f,p,lamb,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any


In [12]:
cost=[ [0	-200	-100	-1200 -2000],
    [0	-400	-100	-1200 -2000],
    [0	-600	-100	-1200 -2000],
    [0	-800	-100	-1200 -2000],
    [0	-1000	-100	-1200 -2000],
    [0	-1200	-100	-1200 -2000],
    [0	-1400	-100	-1200 -2000],
    [0	-1600	-100	-1200 -2000],
    [0	-1800	-100	-1200 -2000],
    [0	-2000	-100	-1200 -2000],
    [0  -2400   -100    -1200 -2000],
    [0  -2800   -100    -1200 -2000],
    [0  -3200   -100    -1200 -2000]
] 



for units in [60,50,40,30,25,20]
    for K in 2:2:convert(Int64, floor((units+1)/2))
        
        global NumberUnits=convert(Int64,units)
        global limit=convert(Int64, K)
    for c in cost
    tick()
   print(units)  
   print("K : ")
    println(K)
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
     println([normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty])   
     for lambda in 1:1:1
        global  lamb=lambda
     Q,V=ValueIteration(.95)
         println(isnan(Q[1,1,1]))
       if sum(isnan(Q[1,1,1]))>0.0
            continue
        end

trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
 for k in 1:simsteps
    a =decison(Q,s)
    r2=0.0
    s1=repeat(1:1,NumberUnits)
              
# Here we are calculating the "true" system-level rewards, following formulation (3) in original paper
# i.e., adding up reward of each component, and decide to add setup cost or not depending on system-level condition 
# In contrast, the policy we are using is obtained only at component level          
#     print(a)
global failed_components = 0
    for i in 1:NumberUnits
#             print(generative(s[i],a[i],234))
         s1[i],r[i]=generative(s[i],a[i],i,234);
          r2+=r[i]
         if s[i] == Number_level
            failed_components += 1
         end
            end 
#     println(r2)
#     println(s)

    if (Number_level in s) || (2 in a) 
        # if any replacement
            r2+=setup_cost
                    end
    if failed_components >= limit
        r2+=system_penalty
    end
    s=s1;
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
#     println(rk)
    results[h]=rk
#     println(results)
end
println(units)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,limit,normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty,lamb,mean(results),std(results)])
        
tock()
end
    println("-------------------------")
end
end
end
end

60K : 2

[ Info:  started timer at: 2024-03-31T19:47:35.467



[0, -200, -100, -1200, -2000]
false
60
1
-192260.09046538177
6361.2433817511155
-------------------------
60K : 2
[0, -400, -100, -1200, -2000]


[ Info:          290.7706296s: 4 minutes, 50 seconds, 770 milliseconds
[ Info:  started timer at: 2024-03-31T19:52:26.378


false
60
1
-195830.33453698462
6250.638157649534
-------------------------
60K : 2
[0, -600, -100, -1200, -2000]


...

20K : 10
[0, -2800, -100, -1200, -2000]
false
20
1
-103884.75754796498
4756.789235614432
-------------------------
20K : 10


[ Info:           125.702555s: 2 minutes, 5 seconds, 702 milliseconds
[ Info:  started timer at: 2024-04-02T14:08:38.564


[0, -3200, -100, -1200, -2000]
false
20
1
-110496.16197292219
5023.92635483372
-------------------------


[ Info:          169.5468957s: 2 minutes, 49 seconds, 546 milliseconds


LoadError: ParseError:
[90m# Error @ [0;0m]8;;file://C:/Users/hhu98/Documents/CW-MDP revision/Code/heterogeneous/In[12]#100:1\[90mIn[12]:100:1[0;0m]8;;\
end
[48;2;120;70;70mend[0;0m
[90m└─┘ ── [0;0m[91minvalid identifier[0;0m

In [13]:
df

Row,u,K,n,s,m,f,p,lamb,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,20.0,10.0,0.0,-3200.0,-100.0,-1200.0,-2000.0,1.0,-1.10496e5,5023.93
2,20.0,10.0,0.0,-2800.0,-100.0,-1200.0,-2000.0,1.0,-1.03885e5,4756.79
3,20.0,10.0,0.0,-2400.0,-100.0,-1200.0,-2000.0,1.0,-97187.5,4561.11
4,20.0,10.0,0.0,-2000.0,-100.0,-1200.0,-2000.0,1.0,-90416.0,4342.76
5,20.0,10.0,0.0,-1800.0,-100.0,-1200.0,-2000.0,1.0,-87192.4,4184.22
6,20.0,10.0,0.0,-1600.0,-100.0,-1200.0,-2000.0,1.0,-83851.3,4115.86
7,20.0,10.0,0.0,-1400.0,-100.0,-1200.0,-2000.0,1.0,-80449.4,3956.61
8,20.0,10.0,0.0,-1200.0,-100.0,-1200.0,-2000.0,1.0,-77167.9,3872.84
9,20.0,10.0,0.0,-1000.0,-100.0,-1200.0,-2000.0,1.0,-73722.3,3831.87


In [14]:
using CSV
CSV.write("Heterogeneous Independent policy.csv", df)

"Heterogeneous Independent policy.csv"

In [15]:
#  Q=ValueIteration(s,T,.95,3000)
# for s in 1:Number_level
# p=[exp(0.01*sum(Q[s,1])),exp(0.01*sum(Q[s,2])),exp(0.01*sum(Q[s,3]))]
#     p./=sum(p)
#     println(p)
# end

In [16]:
#= using Plotsa =#

In [17]:
#= for i in [3,5,10,15,20,25,30,40,50,60]
   display( plot(df[df.Units.==i,:].lambda,df[df.Units.==i,:].mean, xtick=df[df.Units.==i,:].lambda, show = true,title=string(i),label=false, xaxis=:log10,rotation=40))
    plot(show=true)
    println(df[df.Units.==i,:][findmax(df[df.Units.==i,:].mean)[2],:])

end =#

In [18]:
#= plot!()
 =#